In [39]:
import glob
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from scipy.io import wavfile # get the api
import math
from scipy.ndimage.filters import gaussian_filter1d
import numpy as np

file_list=glob.glob('wav Files/*.wav')

def periodize(signal,len_needed,current_len):
    signal_expanded=signal
    to_expand=math.ceil(len_needed/current_len)
    for i in range(to_expand-1):
        signal_expanded=np.row_stack((signal_expanded,signal))
    signal_expanded=np.ravel(signal_expanded)
    return signal_expanded[:len_needed]

def zero_interleave(fourier,desiredT,currentT,cutoff):
    fourier_expanded=fourier
    to_expand=math.ceil(desiredT/currentT)
    for i in range(to_expand-1):
        fourier_expanded=np.column_stack((fourier_expanded,np.zeros(length)))
    fourier_expanded=np.ravel(fourier_expanded)
    fourier_expanded=fourier_expanded[:cutoff]
    return fourier_expanded

def with_spectrum(filename,desiredT):
    freqs, raw_data = wavfile.read(filename) # load the data
    raw_data = raw_data.T[0]
    currentT=len(raw_data)/freqs
    len_needed=desiredT*freqs
    raw_data = periodize(raw_data,len_needed,len(raw_data))
    #raw_data = gaussian_filter1d(raw_data, len_needed/100)
    #plt.plot(raw_data,'r')
    fourier = fft(raw_data)
    length = int(len(fourier)/2)
    fourier = fourier[:length]
    fourier = fourier * [2**(-i*i) for i in range(length)]
    #fourier_expanded=zero_interleave(fourier,desiredT,currentT,1200000)
    #plt.plot(abs(fourier),'g')
    #plt.show()
    return (filename,fourier)
def with_lhs(filename,desiredT,all_hyperplanes):
    _, spectra=with_spectrum(filename,desiredT)
    ambient_dimension = len(spectra)
    return (filename,hash_point(spectra,all_hyperplanes,False))

def hash_point(point,all_hyperplanes,is_real):
    if is_real:
        pre_hashed=np.dot(all_hyperplanes,point)
        return list(map(lambda x: x>=0,pre_hashed))
    else:
        pre_hashed=np.dot(all_hyperplanes,point)-point[0]
        hashed=list(map(lambda x : [np.real(x)>=0,np.imag(x)>=0],pre_hashed))
        return [item for sublist in hashed for item in sublist]

def construct_hyperplanes(num_hyperplanes,ambient_dimension,is_real):
    all_hyperplanes=np.random.randn(num_hyperplanes,ambient_dimension-1)
    if (not is_real):
        all_hyperplanes=all_hyperplanes+1j*np.random.randn(num_hyperplanes,ambient_dimension-1)
        all_hyperplanes=np.hstack((np.ones((num_hyperplanes,1)),all_hyperplanes))
    else:
        all_hyperplanes=np.hstack((np.random.randn(num_hyperplanes,1),all_hyperplanes))
    return all_hyperplanes

def swap_and_simplify(filename,res):
    res_as_num=(np.array([int(x) for x in res])*[2**i for i in range(len(res))]).sum()
    return (res_as_num,filename)

num_points=25
_,y=with_spectrum(file_list[0],30)
all_hyperplanes=construct_hyperplanes(int(np.log2(num_points)),len(y),False)
for i in range(num_points):
    filename,res=with_lhs(file_list[i],30,all_hyperplanes)
    res_as_num,_=swap_and_simplify(filename,res)
    print(filename+": "+str(res_as_num))
    
# sc.parallelize(file_list).map(lambda file: with_lhs(file,30,all_hyperplanes))
#                           .map(lambda filename, res: swap_and_simplify(filename,res))
#                          .groupByKey()

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\scipy\io\wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


wav Files\aerosol-can-spray-01.wav: 18
wav Files\aerosol-can-spray-02.wav: 137
wav Files\aerosol-can-spray-03.wav: 18
wav Files\aerosol-can-spray-04.wav: 235
wav Files\aerosol-can-spray-05.wav: 237
wav Files\aerosol-can-spray-06.wav: 18
wav Files\air-conditioner-01.wav: 214
wav Files\air-conditioner-02.wav: 44
wav Files\aircraft-airliner-landing-01.wav: 237
wav Files\aircraft-cabin-ambience-01.wav: 139
wav Files\aircraft-cabin-ambience-02.wav: 18
wav Files\aircraft-exterior-boarding-01.wav: 139
wav Files\aircraft-exterior-flying-steady-01.wav: 169
wav Files\aircraft-large-airliner-fly-by-01.wav: 2
wav Files\aircraft-large-airliner-fly-by-02.wav: 18
wav Files\aircraft-large-airliner-fly-over-01.wav: 18
wav Files\aircraft-large-airliner-fly-over-02.wav: 2
wav Files\aircraft-large-airliner-fly-over-03.wav: 147
wav Files\aircraft-large-airliner-lift-off-01.wav: 18
wav Files\aircraft-large-airliner-lift-off-02.wav: 3
wav Files\aircraft-large-airliner-take-off-01.wav: 171
wav Files\aircraft-